# Text generation with Markov Chains

In this notebook we are going to look at how we can generate text using Markov Chains. Markov Chains give us a model of sequential data based on the probabilties of what item will follow another (or series of other items). 

Markov Chains are the simplest way that we can make a *generative model* of our text data. They are a bit old fashioned by the standards of todays modern machine learning systems, but they are fast and easy to experiment with, and will serve as a good foundation for when we come learn how to generate text with recurrent neural networks later in the term. There is a nice [visual expainer of Markov Chains here](https://setosa.io/ev/markov-chains/).

First we will need to install the Python library `markovify` that implements our Markov Chains for us:



In [ ]:
%pip install markovify

Now lets import the libraries we will need. `os` will allow us to navigate the file system of whatever PC's Operating System (OS) we are using to load many files at once. We will use `csv` to load in `.csv` (comma separated values) and `.tsv` (tab separated values). `markovify` is the library we will use for performing our text generation.

In [ ]:
import os
import csv
import markovify

## Load files 

Here we will three different util functions for loading text data into our code. Let's just run all of these cells to define the functions now, so we can reuse them later when we start experimenting with different datasets.

The first one just loads the data from a single text file:

In [ ]:
def load_single_text_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        corpus = f.read()
        return corpus

This function loads the data from all the text files in a folder or it's subfolders. There is an optional parameter called `max_files` that you can adjust if you need to reduce the number of files if you are having memory issues.

In [ ]:
def load_all_text_files_in_folder(path, max_files = 10000):
    corpus = ''
    # Find all files in the folder or subfolders
    for root, _, files in os.walk(path):
        for i, file in enumerate(files):
            # If the file is a text file
            if file.endswith(".txt") and i <= max_files:
                # Open the file and add the text to the corpus
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                    # Add text from file
                    corpus += text
                    # Add new line
                    corpus += '\n'
    return corpus

This function loads in text data from a `.csv` or `.tsv` file. For a csv file (comma seperated values) we would pass in a comma in a string variable `','` as the delimeter parameter, for a tsv file (tab separated values) we would pass in a tab character in a string variable `' '` as the delimeter. 

The `col_to_extract` parameter tells us which column our text data is stored in. Here we would pass in the index of that column (remembering that we always count from zero in computers). 

In [ ]:
def load_text_from_csv(csv_file, col_to_extract, delimeter):
    corpus = ''
    # Open csv file
    with open(csv_file, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=delimeter)
        for row in reader:
            # Check to see there is a column where we want to extract
            if len(row) >= col_to_extract:  
                # Get text from the specific column in the row
                text = row[col_to_extract]  
                # Add text to corpus
                corpus += text
                # Add a new line
                corpus += '\n'
    return corpus

## Viewing text as a sequence

Before we generate text using Markov Chains, we are going to look back at some of the visualisation we made in Week 2 that help us understand how text sequences can be represented. 

Lets take a look back at the one hot encoding visualisations we made in Week 2 where we were visualising the occurences of words in our nursery rhyme sequence:

![row row row your boat one hot word vis](../media/im1_row-row-boat-words.png)

For instance if we look at the word ‘row’ the most likely word to follow it is also ‘row’, the second most likely word to follow it is ‘your’. If we think about that in terms of probability, there is a two in three (66.66%) chance that the word following ‘row’ will be ‘row’, whereas there will be a one-in-three (33.33%) chance the following word would be ‘your’. 

However we can also look at multiple words in order before making a prediction with Markov Chains. If we take the sequence ‘row, row’ and count the words that follow that then we only have two options: it is again followed by ‘row’ once, and followed by ‘your’ once. In this case there is a one in two (50%) chance of either word coming next. If we look for the sequence ‘row, row, row’ then there is only one word that follows this sequence, and that is ‘your’. Based on our data there is then a one in one (100%) chance that the word ‘your’ will follow the words ‘row, row, row’. 

If we calculate the probabilities for every transition between every character we get something that looks like this:

![markov chain vis words](../media/markov_chain_visualisation_words.png)

Now let's look at the type of visualisation of the same nursery rhyme, but this time at the character level:

![row row row your boat one hot char vis](../media/im2_row-row-boat-characters.png)

At the character level we can see that the structure is much more complicated. The only letter that repeated itself is now the letter ‘r’ (four times). The other characters that follow ‘r’ is ‘o’ (three times), ‘e’ (two times) and the whitespace character (one time). 

If we calculate the probabilities for every transition between every state we get something that looks like this:

![markov chain vis characters](../media/markov_chain_visualisation_characters.png)

Markov Chains count up all the number of times that one token follows another, be that words or characters, and converts those numbers into probabilities. When using a Markov Chain, we sample our next character based on all of the probabilities of what the next token will be, based on what we have learned from our dataset. 

When we have lots of words in our dataset, then there is a huge amount of possibilities of what word will follow another. Because of that, we tend to need **lots of data** to generate text avoids repetition. When using characters, we have a much smaller dictionary of possible values (at least in English) so when building a Markov Chain on characters we tend to get a lot of nonsense words, unless once again, we have **lots of data**.

## Generating text using Markov Chains

Now we are going make some generative text models using Markov Chains. We will first use our Wikipedia dataset that we have scraped in the other notebook. If you have not worked through the `webscrape-wikipedia` notebook, then go and do that now before continuing any further. 

We will use a single string variable called `corpus` to store our text data for generation. Later on you can change the variable `data_path` to load in a new dataset. You may also need to change the function `load_single_text_file` to [one of the other data loading functions](#load-files):

<a id='load-corpus'></a>

In [ ]:
data_path = '../data/my-data/my-wikipedia-text.txt'
corpus = load_single_text_file(data_path)

### Generating text at the character level

Now lets look at a block of code that is going to generat text for us at the character level. How does it look? Is it generating anything coherent? Try changing the `state_size` which will configure for us how many characters in a sequence we are looking for before the character we are trying predicting probabilities for. The bigger the `state_size` value, the more context the model has for making predictions. **Tip:** You can make this a very high number (500+) when working with characters, though the bigger you make it the slower it will be to calculate the probabilities for the model. 

In [ ]:
corpus_char_list = [ list(word) for word in corpus ]
chain = markovify.Chain(corpus_char_list, state_size=5)

new_str = ''
for i in range(500):
    example_output = chain.walk()
    new_str += example_output[0]

print(new_str)

### Generating text at the word level

Now lets look at a block of code that is going to generat text for us at the word level. See the difference in coherence when generating words compared to the character level generation. We are probably going to get generations that are on the face of it more legible, as we have actual words! However with a small dataset we may end up with quite repetitive predictions. Try increasing the `state_size` parameter this time (in increments of 1) and see how that effects the generations:

In [ ]:
text_model = markovify.Text(corpus, state_size=1)

for i in range(5):
    print(text_model.make_sentence())

## Tasks 

**Task 1:** Once you have run through [character level generation](#generating-text-at-the-character-level) and [word level generation](#generating-text-at-the-word-level) cells a few times, experimenting with the parameters and seeing how that effects the results, go back to the `webscrape-wikipedia` notebook and add some more web pages to the file `data/my-data/my-wikipedia-text.txt`. Remember this appends text to our file so we can keep running it to add more data to the file!

**Task 2:** Download the [Wikipedia Sentences dataset](https://www.kaggle.com/datasets/mikeortman/wikipedia-sentences) and put it into the folder `data/my-data/`. This dataset contains 7.8 million sentences extracted from [the wikipedia data dump](https://dumps.wikimedia.org/enwiki/). This dataset is one text folder so all you will need to do is change the `data_path` variable to the new file in the [cell that loads the text corpus](#load-corpus).

With this larger dataset, experiment again with the `state_size` variables in both the [character level generation](#generating-text-at-the-character-level) and [word level generation](#generating-text-at-the-word-level) cells. How does changing these variables change the results with a much bigger dataset?

**Task 3:** Try some other datasets! Why not try running this code with the nursery rhymes dataset we have been working with that is in `data/nursery-rhymes`. You will need to use [one of the different functions](#load-files) for loading text from multiple files in the the [cell that loads the text corpus](#load-corpus).
As well as the nursery rhymes dataset, there is also [dataset of limericks](https://git.arts.ac.uk/tbroad/limerick-dataset), a [dataset of haikus](https://git.arts.ac.uk/tbroad/haiku-dataset), and a [dataset of EPL fan chants](https://git.arts.ac.uk/tbroad/SFW-EPL-fan-chants-dataset) that are all in the same format as the nursery rhymes dataset. 

**Task 4:** Try loading in a dataset that is stored as a `.csv` or `.tsv`, like the song lyrics examples from last week (`data/TaylorSwift.csv` & `data/tom_waits.tsv`). Again, you will need to use [one of the different functions](#load-files) for loading text from multiple files in the the [cell that loads the text corpus](#load-corpus).

**Task 5:** Look for some more datasets online! Try searching [Kaggle](https://www.kaggle.com/) for datasets that contain text in either a `.txt`, `.csv` or `.tsv` format to use in this notebook. Post the interesting datasets you have found on the [miro board](https://miro.com/app/board/uXjVNXSvetU=/?share_link_id=389409070590).

**Bonus task:** Can you write some code that saves the generated text to a file so you can save it? Refer to the `webscrape-wikipedia` notebook for how to repeatedly write text to the same file by appending to it.